# This Python script provides examples on using the E*TRADE API endpoints
## From:  https://developer.etrade.com/support/downloads

In [1]:
"""This Python script provides examples on using the E*TRADE API endpoints"""
# From:  https://developer.etrade.com/support/downloads
from __future__ import print_function
import webbrowser
import json
import logging
import configparser
import sys
import requests
from rauth import OAuth1Service
from logging.handlers import RotatingFileHandler
from accounts.accounts import Accounts
from market.market import Market
from options.options import Options
from urllib.parse import urlencode
import xml.etree.ElementTree as ET

In [5]:
##### loading configuration file
config = configparser.ConfigParser()

# config.read('C:/Users/jjel0/OneDrive/Data_Analytics/repositories/config_sandbox.ini')
config.read('C:/Users/jjel0/OneDrive/Data_Analytics/repositories/config_live.ini')

# logger settings
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)
handler = RotatingFileHandler("python_client.log", maxBytes=5*1024*1024, backupCount=3)
FORMAT = "%(asctime)-15s %(message)s"
fmt = logging.Formatter(FORMAT, datefmt='%m/%d/%Y %I:%M:%S %p')
handler.setFormatter(fmt)
logger.addHandler(handler)

def oauth():
    """Allows user authorization for the sample application with OAuth 1"""
    etrade = OAuth1Service(
        name = "etrade",
        consumer_key = config["DEFAULT"]["CONSUMER_KEY"],
        consumer_secret = config["DEFAULT"]["CONSUMER_SECRET"],
        request_token_url = "https://api.etrade.com/oauth/request_token",
        access_token_url = "https://api.etrade.com/oauth/access_token",
        authorize_url = "https://us.etrade.com/e/t/etws/authorize?key={}&token={}",
        base_url = "https://api.etrade.com")
    
    menu_items = {"1": "Sandbox Consumer Key",
                  "2": "Live Consumer Key",
                  "3": "Exit"}
    while True:
        print("")
        options = menu_items.keys()
        for entry in options:
            print(entry + ")\t" + menu_items[entry])
        selection = input("Please select Consumer Key Type: ")
        if selection == "1":
            base_url = config["DEFAULT"]["SANDBOX_BASE_URL"]
            break
        elif selection == "2":
            base_url = config["DEFAULT"]["PROD_BASE_URL"]
            break
        elif selection == "3":
            break
        else:
            print("Unknown Option Selected!")
    print("")

    # Step 1: Get OAuth 1 request token and secret 
    request_token, request_token_secret = etrade.get_request_token(
        params = {"oauth_callback": "oob", "format": "json", "range": "1y", "interval": "day"})

    # Step 2: Go through the authentication flow. Login to E*TRADE.
    # After you login, the page will provide a verification code to enter.
    authorize_url = etrade.authorize_url.format(etrade.consumer_key, request_token)
    webbrowser.open(authorize_url)
    text_code = input("Please accept agreement and enter verification code from browser: ")

    # Step 3: Exchange the authorized request token for an authenticated OAuth 1 session
    session = etrade.get_auth_session(request_token,
                                  request_token_secret,
                                  params={"oauth_verifier": text_code})

    main_menu(session, base_url)


def main_menu(session, base_url):
    """
    Provides the different options for the sample application: Market Quotes, Account List, Historical Data

    :param session: authenticated session
    """

    menu_items = {
        "1": "Market Quotes",
        "2": "Account List",
        "3": "Options Chain",
        "4": "Options Expire Dates",
        "5": "Historical Market Data",  # New option
        "6": "Exit"
    }

    while True:
        print("")
        options = menu_items.keys()
        for entry in options:
            print(entry + ")\t" + menu_items[entry])
        selection = input("Please select an option: ")
        if selection == "1":
            market = Market(session, base_url)
            market.quotes()
        elif selection == "2":
            accounts = Accounts(session, base_url)
            accounts.account_list()
        elif selection == "3":
            options = Options(session, base_url)
            options.OptionsQuotes()
        elif selection == "4":
            options = Options(session, base_url)
            options.OptionsExpDate()
        elif selection == "5":  # New option for historical data
            symbol = input("Enter the stock symbol (e.g., AAPL): ")
            range = input("Enter the time range (e.g., 1d, 5d, 1y): ")
            interval = input("Enter the interval (e.g., day, hour): ")
            get_historical_data(session, base_url, symbol, range, interval)
        elif selection == "6":
            break
        else:
            print("Unknown Option Selected!")

def get_historical_data(session, base_url, symbol, range, interval):
    """
    Fetch historical market data.
    :param session: Authenticated session
    :param base_url: API base URL
    :param symbol: Stock symbol
    :param startDate: start date (MMDDYYYY)
    :param endDate: end date (MMDDYYYY)
    """
    url = f"{base_url}/v1/market/quote/{symbol}"
    params = {"range": range, "interval": interval}
    headers = {"Accept": "application/json"}  # Request JSON format explicitly

    response = session.get(url, params=params, headers=headers)

    print(f"Request URL: {response.url}")  # Log the full request URL
    print(f"Response Status Code: {response.status_code}")  # Log the response status

    if response.status_code == 200:
        # Try to parse JSON
        try:
            data = response.json()
            print(json.dumps(data, indent=4))  # Pretty print JSON response
        except json.JSONDecodeError:
            # Handle XML fallback
            print("Response returned as XML. Parsing XML response.")
            try:
                root = ET.fromstring(response.text)
                # Extract relevant data from the XML tree
                for quote in root.findall(".//QuoteData/All"):
                    print(f"Symbol: {symbol}")
                    print(f"Last Trade: {quote.find('lastTrade').text}")
                    print(f"Open: {quote.find('open').text}")
                    print(f"Close: {quote.find('previousClose').text}")
                    print(f"High: {quote.find('high').text}")
                    print(f"Low: {quote.find('low').text}")
            except ET.ParseError as e:
                print(f"XML Parse Error: {e}")
    else:
        print(f"Error fetching historical data: {response.status_code} - {response.text}")

if __name__ == "__main__":
    oauth()



1)	Sandbox Consumer Key
2)	Live Consumer Key
3)	Exit


Please select Consumer Key Type:  2


Please accept agreement and enter verification code from browser:  L8J4B



1)	Market Quotes
2)	Account List
3)	Options Chain
4)	Options Expire Dates
5)	Historical Market Data
6)	Exit


Please select an option:  5
Enter the stock symbol (e.g., AAPL):  aapl
Enter the time range (e.g., 1d, 5d, 1y):  1y
Enter the interval (e.g., day, hour):  day


Request URL: https://api.etrade.com/v1/market/quote/aapl?range=1y&interval=day&oauth_consumer_key=2b1cc0f122bcbd50d64274f3ef0d60ae&oauth_nonce=d2603e37ae3c2969720897303dbb61697d3502dc&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1736461158&oauth_token=VVm3VwkaBHaEzYWbtdGpNwX1TLRrUMTBWnR9WwUitCk%3D&oauth_version=1.0&oauth_signature=JHQFoiNRLmNHXHjJ9X7Eb0iZf%2F4%3D
Response Status Code: 200
{
    "QuoteResponse": {
        "QuoteData": [
            {
                "dateTime": "19:59:51 EST 01-07-2025",
                "dateTimeUTC": 1736297991,
                "quoteStatus": "CLOSING",
                "ahFlag": "true",
                "hasMiniOptions": false,
                "All": {
                    "adjustedFlag": false,
                    "ask": 242.73,
                    "askSize": 0,
                    "askTime": "19:59:51 EST 01-07-2025",
                    "bid": 242.71,
                    "bidExchange": "",
                    "bidSize": 0,
                    "bid

Please select an option:  6


In [ ]:
##### loading configuration file
config = configparser.ConfigParser()

# config.read('C:/Users/jjel0/OneDrive/Data_Analytics/repositories/config_sandbox.ini')
config.read('C:/Users/jjel0/OneDrive/Data_Analytics/repositories/config_live.ini')

# logger settings
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)
handler = RotatingFileHandler("python_client.log", maxBytes=5*1024*1024, backupCount=3)
FORMAT = "%(asctime)-15s %(message)s"
fmt = logging.Formatter(FORMAT, datefmt='%m/%d/%Y %I:%M:%S %p')
handler.setFormatter(fmt)
logger.addHandler(handler)

def oauth():
    """Allows user authorization for the sample application with OAuth 1"""
    etrade = OAuth1Service(
        name = "etrade",
        consumer_key = config["DEFAULT"]["CONSUMER_KEY"],
        consumer_secret = config["DEFAULT"]["CONSUMER_SECRET"],
        request_token_url = "https://api.etrade.com/oauth/request_token",
        access_token_url = "https://api.etrade.com/oauth/access_token",
        authorize_url = "https://us.etrade.com/e/t/etws/authorize?key={}&token={}",
        base_url = "https://api.etrade.com")
    
    menu_items = {"1": "Sandbox Consumer Key",
                  "2": "Live Consumer Key",
                  "3": "Exit"}
    while True:
        print("")
        options = menu_items.keys()
        for entry in options:
            print(entry + ")\t" + menu_items[entry])
        selection = input("Please select Consumer Key Type: ")
        if selection == "1":
            base_url = config["DEFAULT"]["SANDBOX_BASE_URL"]
            break
        elif selection == "2":
            base_url = config["DEFAULT"]["PROD_BASE_URL"]
            break
        elif selection == "3":
            break
        else:
            print("Unknown Option Selected!")
    print("")

    # Step 1: Get OAuth 1 request token and secret 
    request_token, request_token_secret = etrade.get_request_token(
        # params = {"oauth_callback": "oob", "format": "json", "range": "1y", "interval": "day"})
        params = {"oauth_callback": "oob", "format": "json"}) #, "startDate": startDate, "endDate": endDate})

    # Step 2: Go through the authentication flow. Login to E*TRADE.
    # After you login, the page will provide a verification code to enter.
    authorize_url = etrade.authorize_url.format(etrade.consumer_key, request_token)
    webbrowser.open(authorize_url)
    text_code = input("Please accept agreement and enter verification code from browser: ")

    # Step 3: Exchange the authorized request token for an authenticated OAuth 1 session
    session = etrade.get_auth_session(request_token,
                                  request_token_secret,
                                  params={"oauth_verifier": text_code})

    main_menu(session, base_url)


def main_menu(session, base_url):
    """
    Provides the different options for the sample application: Market Quotes, Account List, Historical Data

    :param session: authenticated session
    """

    menu_items = {
        "1": "Market Quotes",
        "2": "Account List",
        "3": "Options Chain",
        "4": "Options Expire Dates",
        "5": "Historical Market Data",  # New option
        "6": "Exit"
    }

    while True:
        print("")
        options = menu_items.keys()
        for entry in options:
            print(entry + ")\t" + menu_items[entry])
        selection = input("Please select an option: ")
        if selection == "1":
            market = Market(session, base_url)
            market.quotes()
        elif selection == "2":
            accounts = Accounts(session, base_url)
            accounts.account_list()
        elif selection == "3":
            options = Options(session, base_url)
            options.OptionsQuotes()
        elif selection == "4":
            options = Options(session, base_url)
            options.OptionsExpDate()
        elif selection == "5":  # New option for historical data
            symbol = input("Enter the stock symbol (e.g., AAPL): ")
            # range = input("Enter the time range (e.g., 1d, 5d, 1y): ")
            # interval = input("Enter the interval (e.g., day, hour): ")
            # get_historical_data(session, base_url, symbol, range, interval)
            startDate = input("Enter the start date (MMDDYYYY): ")
            endDate = input("Enter the end date (MMDDYYYY): ")
            get_historical_data(session, base_url, symbol, startDate, endDate)
        elif selection == "6":
            break
        else:
            print("Unknown Option Selected!")

# def get_historical_data(session, base_url, symbol, range, interval):
def get_historical_data(session, base_url, symbol, startDate, endDate):
    """
    Fetch historical market data.
    :param session: Authenticated session
    :param base_url: API base URL
    :param symbol: Stock symbol
    :param startDate: start date (MMDDYYYY)
    :param endDate: end date (MMDDYYYY)
    """
    url = f"{base_url}/v1/market/quote/{symbol}"
    # params = {"range": range, "interval": interval}
    params = {"startDate": startDate, "endDate": endDate}
    headers = {"Accept": "application/json"}  # Request JSON format explicitly

    response = session.get(url, params=params, headers=headers)

    print(f"Request URL: {response.url}")  # Log the full request URL
    print(f"Response Status Code: {response.status_code}")  # Log the response status
    print(startDate)
    print(endDate)
    if response.status_code == 200:
        # Try to parse JSON
        try:
            data = response.json()
            print(json.dumps(data, indent=4))  # Pretty print JSON response
        except json.JSONDecodeError:
            # Handle XML fallback
            print("Response returned as XML. Parsing XML response.")
            try:
                root = ET.fromstring(response.text)
                # Extract relevant data from the XML tree
                for quote in root.findall(".//QuoteData/All"):
                    print(f"Symbol: {symbol}")
                    print(f"Last Trade: {quote.find('lastTrade').text}")
                    print(f"Open: {quote.find('open').text}")
                    print(f"Close: {quote.find('previousClose').text}")
                    print(f"High: {quote.find('high').text}")
                    print(f"Low: {quote.find('low').text}")
            except ET.ParseError as e:
                print(f"XML Parse Error: {e}")
    else:
        print(f"Error fetching historical data: {response.status_code} - {response.text}")

if __name__ == "__main__":
    oauth()



1)	Sandbox Consumer Key
2)	Live Consumer Key
3)	Exit


Please select Consumer Key Type:  2


Please accept agreement and enter verification code from browser:  T8CJL



1)	Market Quotes
2)	Account List
3)	Options Chain
4)	Options Expire Dates
5)	Historical Market Data
6)	Exit


Please select an option:  5
Enter the stock symbol (e.g., AAPL):  AAPL
Enter the start date (MMDDYYYY):  12012024
Enter the end date (MMDDYYYY):  01072024


Request URL: https://api.etrade.com/v1/market/quote/AAPL?startDate=12012024&endDate=01072024&oauth_consumer_key=2b1cc0f122bcbd50d64274f3ef0d60ae&oauth_nonce=21c4c849bab693d75b1cb2c3e33ac80461d6671c&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1736462039&oauth_token=ip7rPK6ClpoKhvi5go5nzprWwzvE7AKMIqQvkHtacYM%3D&oauth_version=1.0&oauth_signature=44pXP%2FOFYGXjqUGYr37Zz1SDFbc%3D
Response Status Code: 200
12012024
01072024
{
    "QuoteResponse": {
        "QuoteData": [
            {
                "dateTime": "19:59:51 EST 01-07-2025",
                "dateTimeUTC": 1736297991,
                "quoteStatus": "CLOSING",
                "ahFlag": "true",
                "hasMiniOptions": false,
                "All": {
                    "adjustedFlag": false,
                    "ask": 242.73,
                    "askSize": 0,
                    "askTime": "19:59:51 EST 01-07-2025",
                    "bid": 242.71,
                    "bidExchange": "",
                    "bidSi